In [9]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")
llm

Ollama()

In [10]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("abch.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'Adobe PDF Library 24.2.23', 'creator': 'Acrobat PDFMaker 24 for Word', 'creationdate': '2024-06-20T09:32:35-04:00', 'author': 'National Center for Health Statistics', 'category': 'Guidelines', 'company': 'AHIMA', 'contenttypeid': '0x010100F75BD4E2CB5CF94386DDE3793454E76D', 'keywords': 'ICD-10-CM 2018 guidelines', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_actionid': '8136f468-cb46-4293-b85c-dee35068ff72', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_contentbits': '0', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_enabled': 'true', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_method': 'Privileged', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_name': '7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_setdate': '2021-07-02T00:58:31Z', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_siteid': '9ce70869-60db-44fd-abe8-d2767077fc8f', 'moddate': '2024-06-20T09:35:39-04:00', 'sourcemodified': 'D:2024062

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'producer': 'Adobe PDF Library 24.2.23', 'creator': 'Acrobat PDFMaker 24 for Word', 'creationdate': '2024-06-20T09:32:35-04:00', 'author': 'National Center for Health Statistics', 'category': 'Guidelines', 'company': 'AHIMA', 'contenttypeid': '0x010100F75BD4E2CB5CF94386DDE3793454E76D', 'keywords': 'ICD-10-CM 2018 guidelines', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_actionid': '8136f468-cb46-4293-b85c-dee35068ff72', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_contentbits': '0', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_enabled': 'true', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_method': 'Privileged', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_name': '7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_setdate': '2021-07-02T00:58:31Z', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_siteid': '9ce70869-60db-44fd-abe8-d2767077fc8f', 'moddate': '2024-06-20T09:35:39-04:00', 'sourcemodified': 'D:2024062

In [12]:
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'producer': 'Adobe PDF Library 24.2.23', 'creator': 'Acrobat PDFMaker 24 for Word', 'creationdate': '2024-06-20T09:32:35-04:00', 'author': 'National Center for Health Statistics', 'category': 'Guidelines', 'company': 'AHIMA', 'contenttypeid': '0x010100F75BD4E2CB5CF94386DDE3793454E76D', 'keywords': 'ICD-10-CM 2018 guidelines', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_actionid': '8136f468-cb46-4293-b85c-dee35068ff72', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_contentbits': '0', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_enabled': 'true', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_method': 'Privileged', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_name': '7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_setdate': '2021-07-02T00:58:31Z', 'msip_label_7b94a7b8-f06c-4dfe-bdcc-9b548fd58c31_siteid': '9ce70869-60db-44fd-abe8-d2767077fc8f', 'moddate': '2024-06-20T09:35:39-04:00', 'sourcemodified': 'D:2024062

In [13]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [15]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:100],OllamaEmbeddings())

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

In [17]:
retriever = db.as_retriever(search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B5C71CAC90>, search_kwargs={'k': 3})

In [18]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [19]:
response = retrieval_chain.invoke({"input": " What are Endocrine, Nutritional, and Metabolic Diseases codes"})

response['answer']

'Based on the provided context, Endocrine, Nutritional, and Metabolic Diseases codes in ICD-10-CM can be identified as follows:\n\n1. E00-E39: Endocrine diseases, including diabetes mellitus (E10-E14), thyroid disorders (E03-E05), and adrenal insufficiency (E28).\n2. E40-E46: Nutritional deficiency diseases, including scurvy (E40), beriberi (E41), and pellagra (E42).\n3. E50-E89: Metabolic disorders, including obesity (E66), hypoglycemia (E67), and non-alcoholic fatty liver disease (E78).\n\nNote that these codes are located in the "Endocrine, Nutritional, and Metabolic Diseases" section of the ICD-10-CM index. The specific codes and their accompanying descriptions can be found in the Tabular List of ICD-10-CM, which is organized by body system or condition.'